In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

## Test Data Preparation
In this notebook, we will prepare the test.csv in the same way we did for train.csv, and write cleaned test data to csv for final predictions

### Data Cleaning
Most of these steps will be repeated from our EDA and Cleaning notebook and Preprocessing and Feature Engineering notebook

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

In [3]:
# read in test and train data
data = pd.read_csv('../datasets/test.csv')
train_data = pd.read_csv('../datasets/train_clean_fe.csv')

In [4]:
# Remove Unnamed column
train_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
train_data.columns

Index(['id', 'lot_frontage', 'overall_qual', 'bsmtfin_sf_1', 'total_bsmt_sf',
       '1st_flr_sf', 'gr_liv_area', 'full_bath', 'totrms_abvgrd', 'fireplaces',
       'garage_area', 'saleprice', 'porch_sf', 'house_remod_sold',
       'house_age_sold', 'kitchen_qual_Fa', 'kitchen_qual_Gd',
       'kitchen_qual_TA', 'neighborhood_Blueste', 'neighborhood_BrDale',
       'neighborhood_BrkSide', 'neighborhood_ClearCr', 'neighborhood_CollgCr',
       'neighborhood_Crawfor', 'neighborhood_Edwards', 'neighborhood_Gilbert',
       'neighborhood_Greens', 'neighborhood_GrnHill', 'neighborhood_IDOTRR',
       'neighborhood_Landmrk', 'neighborhood_MeadowV', 'neighborhood_Mitchel',
       'neighborhood_NAmes', 'neighborhood_NPkVill', 'neighborhood_NWAmes',
       'neighborhood_NoRidge', 'neighborhood_NridgHt', 'neighborhood_OldTown',
       'neighborhood_SWISU', 'neighborhood_Sawyer', 'neighborhood_SawyerW',
       'neighborhood_Somerst', 'neighborhood_StoneBr', 'neighborhood_Timber',
       'neighbor

In [6]:
# check missing values
data.isna().sum()

Id                   0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       160
Lot Area             0
Street               0
Alley              820
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type         1
Mas Vnr Area         1
Exter Qual           0
Exter Cond           0
Foundation           0
Bsmt Qual           25
Bsmt Cond           25
Bsmt Exposure       25
BsmtFin Type 1      25
BsmtFin SF 1         0
BsmtFin Type 2      25
BsmtFin SF 2         0
Bsmt Unf SF          0
Total Bsmt SF        0
Heating              0
Heating QC           0
Central Air          0
Electrical 

In [7]:
# change column names to snake case
data.columns = data.columns.str.lower()
data.columns = [column.replace(' ', '_') for column in data.columns]

In [8]:
# identify columns that have greater than 1/3 of their values missing
missing_columns = [column for column in data if data.loc[:,column].isna().sum() > 684]
missing_columns

['alley', 'pool_qc', 'fence', 'misc_feature']

In [9]:
# drop missing columns
data.drop(columns = missing_columns, inplace=True)

In [10]:
# impute neighborhood mean into missing values for lot_frontage
lot_means = data.groupby('neighborhood')['lot_frontage'].mean()

for i in range(len(data)):
    if np.isnan(data.loc[i, 'lot_frontage']):
        data.loc[i, 'lot_frontage'] = lot_means[data.loc[i, 'neighborhood']]

In [11]:
# fill remaining lot_frontage values with overall mean
lot_mean = np.mean(data['lot_frontage'])
data['lot_frontage'].fillna(lot_mean, inplace=True)

In [12]:
# mas_vnr_area seems to have too many zeroes to be useful. garage_area seems more related. remove mas_vnr_area and mas_vnr_type
data.drop(columns = ['mas_vnr_type', 'mas_vnr_area'], inplace=True)

In [13]:
# select only columns we had at the end of cleaning out train.csv
data = data[['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'lot_shape', 'land_contour', 'utilities', 'lot_config',
       'land_slope', 'neighborhood', 'condition_1', 'condition_2', 'bldg_type',
       'house_style', 'overall_qual', 'overall_cond', 'year_built',
       'year_remod/add', 'roof_style', 'roof_matl', 'exterior_1st',
       'exterior_2nd', 'exter_qual', 'exter_cond', 'foundation',
       'bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'garage_area', 'paved_drive', 'wood_deck_sf',
       'open_porch_sf', 'enclosed_porch', '3ssn_porch', 'screen_porch',
       'pool_area', 'misc_val', 'mo_sold', 'yr_sold', 'sale_type']]

### Feature Engineering Test Data
Perform steps we used on our train data again to our test data

In [14]:
# Sum wooden_deck_sf, open_porch_sf and screen_porch into 'porch_sf' column, drop old columns
data['porch_sf'] = data['wood_deck_sf'] + data['open_porch_sf'] + data['screen_porch']
data.drop(columns=['open_porch_sf', 'wood_deck_sf', 'screen_porch'], inplace=True)

In [15]:
# Make new column 'house_remod_sold' from year_remod/add - yr_sold
data['house_remod_sold'] = data['yr_sold'] - data['year_remod/add']

In [16]:
# Make new column 'house_age_sold' from year_built - yr_sold, drop old columns
data['house_age_sold'] = data['yr_sold'] - data['year_built']
data.drop(columns=['year_built', 'yr_sold', 'year_remod/add'], inplace=True)

In [17]:
# remove basement 2 data due to very low correlation to price
# these columns seem already well represented by total_bsmt_sf
data.drop(columns=['bsmt_half_bath', 'bsmtfin_sf_2','bsmt_unf_sf'], inplace=True)

In [18]:
# Drop features to reduce potential noise
data.drop(columns = ['kitchen_abvgr', '2nd_flr_sf', 'low_qual_fin_sf', 'enclosed_porch', 'overall_cond', 'misc_val', 'pool_area', 'bedroom_abvgr'], inplace=True)

In [19]:
# drop features with low correlation
data.drop(columns = ['lot_area', 'bsmt_full_bath', 'half_bath', '3ssn_porch', 'pid'], inplace=True)

In [20]:
# drop mo_sold, ms_subclass and bldg_type due to low variation in saleprice by category
data.drop(columns=['bldg_type', 'mo_sold', 'ms_subclass'], inplace=True)

In [21]:
# The kitchen_qual column for test.csv contains a value (Po) that train.csv does not
# This will be replaced with TA (typical)
data['kitchen_qual'][data['kitchen_qual'] == 'Po'] = 'TA'

/home/mark/Python/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
# One hot encode kitchen_qual and neighborhood
data = pd.get_dummies(columns = ['kitchen_qual', 'neighborhood'], data=data, drop_first=True)

In [23]:
# remove other categorical features for now
data = data.select_dtypes(exclude=['object'])

In [24]:
# check for columns in train.csv but not in test data
list(set(train_data.columns) - set(data.columns))

['saleprice', 'neighborhood_GrnHill', 'neighborhood_Landmrk']

In [26]:
# There are houses in two neighborhoods in train.csv which do not exist in test.csv.
# These OHE columns must be removed or predictions can not be made
train_data.drop(columns = ['neighborhood_GrnHill', 'neighborhood_Landmrk'], inplace=True)

In [27]:
# write test and train data to csv for final predictions
data.to_csv('../datasets/test_clean.csv')
train_data.to_csv('../datasets/train_clean_final.csv')